In [41]:
import os
import json
import statistics

In [42]:
def get_statistics(dir_path):
    stats_dict = {
        'measures_at_cut_off': {'rank': [], 'precision': [], 'recall': [], 'f1_score': []},
        'measures_at_max_f1_score': {'rank': [], 'precision': [], 'recall': [], 'f1_score': []}
    }


    for dir in os.listdir(dir_path):
        with open(dir_path+dir+'/results_summary.json') as f:
            data = json.load(f)

            # measures at cut off
            measures_at_cut_off_dict = data['measures_at_cut_off']
            stats_dict['measures_at_cut_off']['rank'].append(measures_at_cut_off_dict['rank'])
            stats_dict['measures_at_cut_off']['precision'].append(measures_at_cut_off_dict['precision'])
            stats_dict['measures_at_cut_off']['recall'].append(measures_at_cut_off_dict['recall'])
            stats_dict['measures_at_cut_off']['f1_score'].append(measures_at_cut_off_dict['f1_score'])

            # measures at max f1-score
            measures_at_max_f1_score = data['measures_at_max_f1_score']
            stats_dict['measures_at_max_f1_score']['rank'].append(measures_at_max_f1_score['rank'])
            stats_dict['measures_at_max_f1_score']['precision'].append(measures_at_max_f1_score['precision'])
            stats_dict['measures_at_max_f1_score']['recall'].append(measures_at_max_f1_score['recall'])
            stats_dict['measures_at_max_f1_score']['f1_score'].append(measures_at_max_f1_score['f1_score'])


    # Get average and standard deviation for each measurement
    print('Statistics at cut_off')
    print('Average Rank:', statistics.mean(stats_dict['measures_at_cut_off']['rank']), 'stdev:', statistics.stdev(stats_dict['measures_at_cut_off']['rank']))
    print('Precision:', statistics.mean(stats_dict['measures_at_cut_off']['precision']), 'stdev:', statistics.stdev(stats_dict['measures_at_cut_off']['precision']))
    print('Recall:', statistics.mean(stats_dict['measures_at_cut_off']['recall']), 'stdev:', statistics.stdev(stats_dict['measures_at_cut_off']['recall']))
    print('F1-Score:', statistics.mean(stats_dict['measures_at_cut_off']['f1_score']), 'stdev:', statistics.stdev(stats_dict['measures_at_cut_off']['f1_score']))

    print()
    print('Statistics at maximum F1-score')
    print('Average Rank:', statistics.mean(stats_dict['measures_at_max_f1_score']['rank']), 'stdev:', statistics.stdev(stats_dict['measures_at_max_f1_score']['rank']))
    print('Precision:', statistics.mean(stats_dict['measures_at_max_f1_score']['precision']), 'stdev:', statistics.stdev(stats_dict['measures_at_max_f1_score']['precision']))
    print('Recall:', statistics.mean(stats_dict['measures_at_max_f1_score']['recall']), 'stdev:', statistics.stdev(stats_dict['measures_at_max_f1_score']['recall']))
    print('F1-Score:', statistics.mean(stats_dict['measures_at_max_f1_score']['f1_score']), 'stdev:', statistics.stdev(stats_dict['measures_at_max_f1_score']['f1_score']))

# Approximate BC by varying the source and target nodes used

In [43]:
all_nodes_dir = 'network_analysis/figures/TUS_all_nodes/'

print('Sources: ALL NODES, Targets: ALL NODES \n')
get_statistics(all_nodes_dir)

Sources: ALL NODES, Targets: ALL NODES 

Statistics at cut_off
Average Rank: 26035 stdev: 0.0
Precision: 0.6219627424620703 stdev: 0.008283149709314696
Recall: 0.6219627424620703 stdev: 0.008283149709314696
F1-Score: 0.6219627424620703 stdev: 0.008283149709314696

Statistics at maximum F1-score
Average Rank: 27793.4 stdev: 2765.021392322309
Precision: 0.6309937354191688 stdev: 0.022605626088718675
Recall: 0.6720952563856347 stdev: 0.05194884818229166
F1-Score: 0.6495183528487658 stdev: 0.020680816467665293


In [44]:
cell_nodes_dir = 'network_analysis/figures/TUS_cell_nodes/'

print('Sources: CELL NODES, Targets: CELL NODES \n')
get_statistics(cell_nodes_dir)

Sources: CELL NODES, Targets: CELL NODES 

Statistics at cut_off
Average Rank: 26539 stdev: 0.0
Precision: 0.5854704397302084 stdev: 0.009335898611253649
Recall: 0.5854704397302084 stdev: 0.009335898611253649
F1-Score: 0.5854704397302084 stdev: 0.009335898611253649

Statistics at maximum F1-score
Average Rank: 22827.6 stdev: 96.0536308527689
Precision: 0.67308804305673 stdev: 0.002084463469805406
Recall: 0.5789592674931233 stdev: 0.003164592400967382
F1-Score: 0.6224831662415277 stdev: 0.0024974607181261067


In [45]:
source_cell_target_all_dir = 'network_analysis/figures/TUS_source_cell_target_all/'

print('Sources: CELL NODES, Targets: ALL NODES \n')
get_statistics(source_cell_target_all_dir)

Sources: CELL NODES, Targets: ALL NODES 

Statistics at cut_off
Average Rank: 26035 stdev: 0.0
Precision: 0.5947685807566737 stdev: 0.006713452073479037
Recall: 0.5947685807566737 stdev: 0.006713452073479037
F1-Score: 0.5947685807566737 stdev: 0.006713452073479037

Statistics at maximum F1-score
Average Rank: 22947.2 stdev: 106.17532670069822
Precision: 0.6676263010820271 stdev: 0.0021204497079254933
Recall: 0.5884463222584981 stdev: 0.0036773679146315484
F1-Score: 0.6255379788679245 stdev: 0.002807573890311432


In [46]:
source_cell_target_all_dir = 'network_analysis/figures/TUS_source_all_target_cell/'

print('Sources: ALL NODES, Targets: CELL NODES \n')
get_statistics(source_cell_target_all_dir)

Sources: ALL NODES, Targets: CELL NODES 

Statistics at cut_off
Average Rank: 26035 stdev: 0.0
Precision: 0.6 stdev: 0.005675499610824288
Recall: 0.6 stdev: 0.005675499610824288
F1-Score: 0.6 stdev: 0.005675499610824288

Statistics at maximum F1-score
Average Rank: 23294.6 stdev: 39.513288903861195
Precision: 0.6673295786373549 stdev: 0.0008129859291119666
Recall: 0.5970885346648742 stdev: 0.0017171815919157952
F1-Score: 0.63025767757547 stdev: 0.0013151732647648828
